# Stellar rotational broadening and RV shifting

The emergent spectrum of a star gets modulated by at least two *extrinsic* parameters: $v\sin{i}$ and $RV$.  Here we show how to modulate a pre-trained cloned model with these two parameters.

In [ ]:
import torch
from blase.emulator import SparseLinearEmulator, SparseLogEmulator
from blase.utils import doppler_grid
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

In [ ]:
wl_lo = 11_000-60
wl_hi = 11_180+60
wavelength_grid = doppler_grid(wl_lo, wl_hi)

We can now use any wavelength grid we like!  We choose a uniform-in-velocity-spacing grid.

In [ ]:
stellar_pretrained_model = torch.load('phoenix_clone_T4700g4p5_prom0p01_11000Ang.pt')
telluric_pretrained_model = torch.load('telfit_clone_temp290_hum040_prom0p01_11000Ang.pt')

In [ ]:
stellar_emulator = SparseLinearEmulator(wavelength_grid, 
                                        init_state_dict=stellar_pretrained_model)

In [ ]:
telluric_emulator = SparseLogEmulator(wavelength_grid, 
                                        init_state_dict=telluric_pretrained_model)

In [ ]:
with torch.no_grad():
    stellar_flux = stellar_emulator.forward().cpu().numpy()
    telluric_transmission = telluric_emulator.forward().cpu().numpy()

In [ ]:
plt.step(wavelength_grid, stellar_flux, label='Pretrained stellar model')
plt.fill_between(wavelength_grid, telluric_transmission, 1.0,
                label='Pretrained telluric model absorption', alpha=0.1, color='k')
plt.xlim(11_080, 11_090)
plt.legend();

Voilá! We successfully read-in our pretrained models!